In [15]:
from pprint import pprint
import numpy as np 

# python-sounddevice の使用例

In [16]:
import sounddevice as sd 

[python-sounddevice](https://python-sounddevice.readthedocs.io/)は，クロスプラットフォームの音響デバイス制御ライブラリ[PortAudio](http://www.portaudio.com/)のPythonバインディング(PythonAPI)です．



##　MicInをリアルタイム処理

In [37]:
duration = 10  # in [s]　callbackを回している間待っている時間．

def printRMS_MicIn(indata: np.ndarray, frames: int, time: CData, status: CallbackFrags) -> None:
    """calcurate RMS(RootMeanSquare) from Mic chunk

    Args:
        indata (np.ndarray): input buffer
        frames (int): number of sample(frame), a.k.a. length of buffer
        time (CData): time of the first sample in the input and time the callback called
        status (CallbackFrags): 入力パッファに割り込み/アンダーフロー/オーバーフロー
    
    No Return, but print RMS value
    """

    # indata.shape = (n_samples, n_channels)
    if status:
        print(status, file=sys.stderr)
    print(np.sqrt(np.mean(indata**2)))

# Streamを開きCallbackを仕掛ける，duration秒待っている間にCallbackが回る，終わったら最後にstreamを閉じる
# sleepがなければ，callbackを待たないので何も処理されない．
# 監視時間を決めたくないなら，とりあえずdurationを待つことにし，with の前に `while true:`かな
with sd.InputStream(channels=1,dtype='float32',
    callback=printRMS_MicIn):
    sd.sleep(int(duration * 1000))  #sleepの引数は秒ではなくミリ秒


0.007086062
0.02902551
0.011275179
0.024600876
0.021741522
0.0103731295
0.0058564837
0.0053720106
0.0051562563
0.0038082178
0.0055557676
0.005952194
0.006113075
0.005873906
0.006322756
0.0048007914
0.0057322765
0.0075412914
0.0077628316
0.0062265717
0.006037567
0.0119456975
0.012630078
0.006645734
0.0055490085
0.0077818157
0.0068162438
0.0070936666
0.0058507277
0.009328963
0.005760982
0.0056672967
0.008148399
0.0070186676
0.006193348
0.0064555085
0.007125645
0.00847808
0.006505611
0.0071803187
0.006262556
0.0066450164
0.005012438
0.0072094845
0.0059828786
0.007452241
0.006465482
0.005523118
0.006177779
0.004927774
0.005095343
0.0076745776
0.0069928365
0.006127313
0.00759033
0.0064180344
0.004767807
0.0041321623
0.0052103046
0.0052086404
0.007959801
0.009173259
0.0068991072
0.0055858367
0.0056026364
0.0048245788
0.005904665
0.008621556
0.0059795817
0.005523822
0.007294004
0.004001203
0.00447898
0.0047964766
0.0046451637
0.007673122
0.008781929
0.006933379
0.0046594865
0.0058121104
0.003

## SpOUTにリアルタイム再生

系列を作ったそばから再生

In [36]:
duration = 3

# ピーポーサイレンを作る
freqs = {"pee":970, "poo":780} # [Hz]
len_p = 0.65 # [s]

sd.default.device = [3,5]
output_device_info = sd.query_devices(device=sd.default.device[1])
fs = int(output_device_info["default_samplerate"])

def SpOut_generatingSiren(outdata, frames, time, status):
    global offset,fs,freqs # callback の間保持している変数
    n_samples, n_channels = outdata.shape
    t = np.arange(offset,offset+n_samples)/fs
    if (offset/fs)%(2*len_p) < len_p:
        f_now = freqs["pee"]
    else:
        f_now = freqs["poo"]
    for k in range(n_channels):
        outdata[:,k] = np.sin(2*np.pi*f_now*t)
    offset += n_samples

offset = 0
with sd.OutputStream(channels=2,dtype='float32',callback=SpOut_generatingSiren):
    sd.sleep(int(duration * 1000))




## MicInをエフェクトかけてSpOut

ヘッドセットMicに入ってきたFM変調音を復調してヘッドセットSpOutとか

In [ ]:
def SpOut_demodulated_MicIn(indata, outdata, frames, time, status):
    n_samples, n_channels = outdata.shape
    outdata[:] = demodulate(indata)

with sd.Stream(channels=2,dtype='float32',callback=SpOut_demodulated_MicIn):
    sd.sleep(int(duration * 1000)) 


## MicInを録音（チャンク処理ではなく，時間を決めて系列化）

In [40]:
duration = 10
fs = 44100
nchannels = 2

data = sd.rec(int(duration * fs), samplerate=fs, channels=2)
sd.wait() # 録音終了待ち

print(data.shape)

(441000, 2)


## 系列をSpOut再生

`sd.play(np.ndarray, fs)`

↑で保存したdataを再生してみよう

In [42]:
sd.play(data,fs)
sd.wait()

## デバイスの選択

In [34]:
device_list = sd.query_devices()
print(device_list)

defaultIn, defaultOut = sd.default.device
print("==Micは{}==".format(defaultIn))
pprint(device_list[defaultIn])
print("==Spは{}==".format(defaultOut))
pprint(device_list[defaultOut])


  0 ambie AM-TW01, Core Audio (1 in, 0 out)
  1 ambie AM-TW01, Core Audio (0 in, 2 out)
  2 W2600, Core Audio (0 in, 2 out)
> 3 Logicool Webcam C925e, Core Audio (2 in, 0 out)
  4 外部マイク, Core Audio (1 in, 0 out)
< 5 外部ヘッドフォン, Core Audio (0 in, 2 out)
  6 Mac miniのスピーカー, Core Audio (0 in, 2 out)
  7 Iriun Webcam Audio, Core Audio (2 in, 0 out)
==Micは3==
{'default_high_input_latency': 0.28925,
 'default_high_output_latency': 0.1,
 'default_low_input_latency': 0.26125,
 'default_low_output_latency': 0.01,
 'default_samplerate': 16000.0,
 'hostapi': 0,
 'max_input_channels': 2,
 'max_output_channels': 0,
 'name': 'Logicool Webcam C925e'}
==Spは5==
{'default_high_input_latency': 0.1,
 'default_high_output_latency': 0.0150625,
 'default_low_input_latency': 0.01,
 'default_low_output_latency': 0.005729166666666666,
 'default_samplerate': 48000.0,
 'hostapi': 0,
 'max_input_channels': 0,
 'max_output_channels': 2,
 'name': '外部ヘッドフォン'}


In [7]:
sd.default.device = [3,2]
defaultIn, defaultOut = sd.default.device
print("==Micは{}==".format(defaultIn))
pprint(device_list[defaultIn])
print("==Spは{}==".format(defaultOut))
pprint(device_list[defaultOut])

==Micは3==
{'default_high_input_latency': 0.28925,
 'default_high_output_latency': 0.1,
 'default_low_input_latency': 0.26125,
 'default_low_output_latency': 0.01,
 'default_samplerate': 16000.0,
 'hostapi': 0,
 'max_input_channels': 2,
 'max_output_channels': 0,
 'name': 'Logicool Webcam C925e'}
==Spは2==
{'default_high_input_latency': 0.1,
 'default_high_output_latency': 0.019166666666666665,
 'default_low_input_latency': 0.01,
 'default_low_output_latency': 0.009833333333333333,
 'default_samplerate': 48000.0,
 'hostapi': 0,
 'max_input_channels': 0,
 'max_output_channels': 2,
 'name': 'W2600'}


In [8]:
pprint(sd.query_hostapis())

({'default_input_device': 0,
  'default_output_device': 1,
  'devices': [0, 1, 2, 3, 4, 5, 6, 7],
  'name': 'Core Audio'},)


In [12]:
hostapis = sd.query_hostapis()
for dicts in hostapis:
    print("{name}対応デバイスは{devices}で".format(name=dicts['name'],devices=dicts['devices']))
    print("デフォルトは In:{defaultIn}, Out:{defaultOut}".format(defaultIn=dicts['default_input_device'],defaultOut=dicts['default_output_device']))


Core Audio対応デバイスは[0, 1, 2, 3, 4, 5, 6, 7]で
デフォルトは In:0, Out:1
